---
* **Universidad:** UOC (Universitat Oberta de Catalunya)
* **Titulación:** Master Ciencia de Datos/TFM
* **Título:** Segmentación de imágenes utilizando Redes Generativas Antagónicas
* **Actividad**: Segmentación partes cuerpo personas con armas cortas utilizando modelos pre entrenados
* **Autor:** Miguel Ponce
* **Correo:** miguelalejoponce@uoc.edu
* **Fecha:** "Enero, 04 de 2023"
* **Salida**: ipynb
---
## Contexto: Detección de personas

* **Contexto:**
Se desea segmentar las personas y armas cortas con los modelos pre entrenados.

* **Problema:** 
Es necesrio realizar la segmentación de las personas armadas con armas cortas, para tener un conjunto de imagenes que luego serán procesadas para ser utilizadas como conjunto de datos para la red GAN.

* **Solucion :** 
Se utilizara los modelos pre entrenados de las etapas anteriores para generar un nuevo conjunto de datos. Para esto se hara uso de los conjuntos de datos tratado con super pixeles ESRGAN y RDN.
* **Entorno:** con GPU.
  

## Origen del la fuente de datos
* **ESRGAN:**
https://github.com/miguelalejo/gan-segmentation-person-gun/blob/main/data/raw/super-resolution-esrgan-personas-armadas.tar.gz

* **RDN:** https://github.com/miguelalejo/gan-segmentation-person-gun/blob/main/data/raw/super-resolution-rdn.tar.gz

## Modelos pre entrenados
* **DETECCIÓN ARMA:**
https://drive.google.com/file/d/1xAbxae-WI2uiTPUOZvakSNhvbkNBc2sT/view

* **SEGMENTACIÓN ARMA:** https://drive.google.com/file/d/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob/view

* **SEGMENTACIÓN PERSONA:** https://github.com/de-code/python-tf-bodypix

# Descarga conjunto de datos

In [ ]:
!wget https://media.githubusercontent.com/media/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-rdn.tar.gz  -O super-resolution-rdn.tar.gz 
!wget https://media.githubusercontent.com/media/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-esrgan-personas-armadas.tar.gz -O super-resolution-esrgan-personas-armadas.tar.gz


--2023-01-05 13:33:03--  https://media.githubusercontent.com/media/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-rdn.tar.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74354007 (71M) [application/octet-stream]
Saving to: ‘super-resolution-rdn.tar.gz’

super-resolution-rd 100%[===================>]  70.91M   226MB/s    in 0.3s    

2023-01-05 13:33:05 (226 MB/s) - ‘super-resolution-rdn.tar.gz’ saved [74354007/74354007]

--2023-01-05 13:33:05--  https://media.githubusercontent.com/media/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-esrgan-personas-armadas.tar.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to

In [ ]:
!tar -xf super-resolution-rdn.tar.gz
!tar -xf super-resolution-esrgan-personas-armadas.tar.gz

# Deteccion Armas Cortas

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-gc0zrmqf
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-gc0zrmqf
  Resolved https://github.com/facebookresearch/detectron2.git to commit 06f2892d2c3125deb207fc61f52484a428d67add
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 5.9 MB/s eta 0:00:0

## Importar Librerias



In [ ]:
import os
import glob
import time
import torch
from pathlib import Path
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_hub as hub
import logging
import torch
from collections import OrderedDict
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.utils.events import EventStorage
from detectron2.modeling import build_model
import detectron2.utils.comm as comm
from detectron2.engine import default_argument_parser, default_setup, default_writers, launch
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    inference_on_dataset,
    print_csv_format,
)

## Mostrar caracteristicas del entorno

In [ ]:
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.13 ; cuda:  cu116
detectron2: 0.6


## Constantes

In [ ]:
# Declaring Constants
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"
TEST_MODE = True
MAX_IMAGES = 30
mainPath = "./"

## Clases


In [ ]:
class DetectImage:
  def __init__(self,sourcePath,outputPath,config,modelName,threshold) -> None:
    self.cfg = config
    print(os.path.join(self.cfg.OUTPUT_DIR, modelName))
    self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, modelName)
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    self.predictor = DefaultPredictor(self.cfg)
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def crop_object(self,imagePath):
    """Crops an object in an image

    Inputs:
      image: PIL image
      box: one box from Detectron2 pred_boxes
    """  
    image = Image.open(imagePath)
    im = cv2.imread(imagePath)
    outputs = self.predictor(im)
    boxes = outputs["instances"].pred_boxes
    # Select 1 box:    
    if list(boxes):
      #print(imagePath)
      #print(boxes,"boxes")
      box = list(boxes)[0].detach().cpu().numpy()         
      x_top_left = box[0]
      y_top_left = box[1]
      x_bottom_right = box[2]
      y_bottom_right = box[3]
      x_center = (x_top_left + x_bottom_right) / 2
      y_center = (y_top_left + y_bottom_right) / 2
      crop_img = image.crop((int(x_top_left), int(y_top_left), int(x_bottom_right), int(y_bottom_right)))
      #plt.figure()
      #plt.axis('off')
      #plt.imshow(crop_img) 
      #plt.show() 
      return True,crop_img,box
    else:
      return False,None,None

  def saveBox(self,box,boxPath):
    with open(boxPath, "w") as f:
      for s in box:
        f.write(str(s) +"\n")
  def readBox(self,boxPath):
    values = []
    with open(boxPath, "r") as f:
      for line in f:
        values.append(float(line.strip()))
    return values
   
  
  def processImages(self):       
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    start = time.time()
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1             
      hasBox,crop_img,box = self.crop_object(imagePath)
      if hasBox:
        imageName = os.path.basename(imagePath)
        print(imageName)
        name = Path(imagePath).stem        
        boxPath = os.path.join(self.outputPath,"boxes",name+".txt")
        cropImagePath = os.path.join(self.outputPath,"crops",imageName)
        crop_img.save(cropImagePath)
        self.saveBox(box,boxPath)
        print("Time Detect Image : %f" % (time.time() - start))

## Configuracion modelos pre entrenados

In [ ]:
from detectron2.engine import DefaultTrainer
instance = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(instance))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(instance)  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Descaga modelo detección arma pre-entrenado

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1" -O ./output/model_final.pth && rm -rf /tmp/cookies.txt

--2023-01-05 13:36:50--  https://docs.google.com/uc?export=download&confirm=t&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1
Resolving docs.google.com (docs.google.com)... 173.194.215.113, 173.194.215.100, 173.194.215.102, ...
Connecting to docs.google.com (docs.google.com)|173.194.215.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cpcj8tvnj3gbt0arnuisrrvp16e94mre/1672925775000/07156903365772177219/*/1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1?e=download&uuid=a97d1b33-936f-4336-8471-e74631eaef6e [following]
--2023-01-05 13:36:50--  https://doc-0g-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cpcj8tvnj3gbt0arnuisrrvp16e94mre/1672925775000/07156903365772177219/*/1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1?e=download&uuid=a97d1b33-936f-4336-8471-e74631eaef6e
Resolving doc-0g-bk-docs.googleusercontent.com (doc-0g-bk-docs.googleusercontent.com)... 142.250.97.1

## Creacion proyecto deteccion arma RDN

In [ ]:
!rm -rf guns-detection-rdn
!mkdir guns-detection-rdn
!mkdir "guns-detection-rdn/crops"
!mkdir "guns-detection-rdn/boxes"
!mkdir "guns-detection-rdn/binmask"
!mkdir "guns-detection-rdn/binredmask"

## Creacion Pipeline red pre entrenada detección arma RDN

In [ ]:
sourcePathRDN = os.path.join(mainPath,"super-resolution-rdn/person")
outputPathRDN = os.path.join(mainPath,"guns-detection-rdn")
detectImage = DetectImage(sourcePathRDN,outputPathRDN,cfg,"model_final.pth",0.85)
detectImage.processImages()

./output/model_final.pth
[01/05 13:37:06 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
./super-resolution-rdn/person/*.jpg


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


armas-299-_jpg.rf.7b79e627eca3de74fa325558ecb7c0a1.jpg
Time Detect Image : 7.001651
gmd_3_frame84_jpg.rf.a07c5f0c12debf8c9ca921839613e496.jpg
Time Detect Image : 7.527903
armas-263-_jpg.rf.69d306c56b8ce31ae3f7c134245a688f.jpg
Time Detect Image : 7.784901
gmd_1_frame194_jpg.rf.502932c0d41979ec30fdc4455a231b6a.jpg
Time Detect Image : 8.024814
mgd_varying_frame147_jpg.rf.739f0e78220469228c7e068326f3dbd7.jpg
Time Detect Image : 8.326728
armas-30-_jpg.rf.59f17a0c15a0e98acabf77885b8c8007.jpg
Time Detect Image : 8.546806
mgd_varying_frame68_jpg.rf.ef2ad947147d790a5c041004724d1067.jpg
Time Detect Image : 8.815249
454744887-1615223010_jpeg_jpg.rf.4c43ec9299cd0504550e013f28a18b96.jpg
Time Detect Image : 9.101267
419159897-1606023748_jpeg_jpg.rf.f1cb74fb2d8d72fc9d9669f8fd602ea3.jpg
Time Detect Image : 9.361582
423637303-1607076161_jpeg_jpg.rf.df58c0cec8182cbd3ee007e32019b2fc.jpg
Time Detect Image : 9.654107
gmd_2_frame38_jpg.rf.8d3dba4f89d4214bd63a95a0f01787ac.jpg
Time Detect Image : 9.898172
mgd

# Segmentacion Armas

## Clases

In [ ]:
class SegmentImage:
  def __init__(self,sourcePath,outputPath,config,modelName,threshold) -> None:
    self.cfg = config
    self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, modelName)
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    self.predictor = DefaultPredictor(self.cfg)
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def crop_shape_object(self,imagePath):
    """Crops an object in an image

    Inputs:
      image: PIL image
      box: one box from Detectron2 pred_boxes
    """  
   
    img= cv2.imread(imagePath)
    outputs= self.predictor(img)
    if outputs['instances']: 
      mask= outputs['instances'].get('pred_masks')    
      mask= mask.to('cpu')    
      num, h, w= mask.shape
      bin_mask= np.zeros((h, w))
      index  = 0
      for m in mask:        
          if index > 0:
            break;
          bin_mask = m.numpy().astype(np.uint8)
          #indices = indices.astype(np.uint8)  #convert to an unsigned byte
          bin_mask*=255
          #print(bin_mask.any(0))
          #print(index)
          index = index +1
          #bin_mask.append(m)
      return True,bin_mask
    return False,None
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1                
      hasMask,bin_mask = self.crop_shape_object(imagePath)
      if hasMask:
        imageName = os.path.basename(imagePath)
        print(imageName)
        imageNamePng = Path(imageName).stem+'.png'
        im = Image.fromarray(bin_mask)
        imagePngPath = os.path.join(self.outputPath,imageNamePng)                        
        im.save(imagePngPath, "PNG")
        print("Time Segment Image : %f" % (time.time() - start))

## Configuracion modelo segementación pre entrenado

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("asl_poly_train",)#Train dataset registered in a previous cell
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00055
cfg.SOLVER.MAX_ITER = 1200 
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # 26 letters plus one super class
cfg.TEST.EVAL_PERIOD = 0 # Increase this number if you want to monitor validation performance during training

PATIENCE = 500 #Early stopping will occur after N iterations of no imporovement in total_loss

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Descaga modelo segemntación arma pre-entrenado

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob" -O ./output/model_final_segmentaion.zip && rm -rf /tmp/cookies.txt

--2023-01-05 13:37:21--  https://docs.google.com/uc?export=download&confirm=t&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob
Resolving docs.google.com (docs.google.com)... 74.125.28.102, 74.125.28.113, 74.125.28.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.28.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tubdn1nrn05cnk6cs6tqqfa8dtga2421/1672925775000/07156903365772177219/*/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob?e=download&uuid=bfc26ae3-ed89-4d33-9ec7-bcd2b7eaf455 [following]
--2023-01-05 13:37:22--  https://doc-08-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tubdn1nrn05cnk6cs6tqqfa8dtga2421/1672925775000/07156903365772177219/*/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob?e=download&uuid=bfc26ae3-ed89-4d33-9ec7-bcd2b7eaf455
Resolving doc-08-bk-docs.googleusercontent.com (doc-08-bk-docs.googleusercontent.com)... 142.250.97.132, 2607

In [ ]:
!unzip -o ./output/model_final_segmentaion.zip -d ./output/

Archive:  ./output/model_final_segmentaion.zip
  inflating: ./output/model_final_segmentaion.pth  


## Creacion Pipeline red pre entrenada segmentación arma RDN

In [ ]:
sourcePathCropsRDN = os.path.join(mainPath,"guns-detection-rdn/crops")
outputPathBinMaskRDN = os.path.join(mainPath,"guns-detection-rdn/binmask")
segmentImage = SegmentImage(sourcePathCropsRDN,outputPathBinMaskRDN,cfg,"model_final_segmentaion.pth",0.60)
segmentImage.processImages()

[01/05 13:37:28 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final_segmentaion.pth ...
armas-299-_jpg.rf.7b79e627eca3de74fa325558ecb7c0a1.jpg
Time Segment Image : 0.152154
gmd_3_frame84_jpg.rf.a07c5f0c12debf8c9ca921839613e496.jpg
Time Segment Image : 0.295555
armas-263-_jpg.rf.69d306c56b8ce31ae3f7c134245a688f.jpg
Time Segment Image : 0.419737
gmd_1_frame194_jpg.rf.502932c0d41979ec30fdc4455a231b6a.jpg
Time Segment Image : 0.531806
mgd_varying_frame147_jpg.rf.739f0e78220469228c7e068326f3dbd7.jpg
Time Segment Image : 0.634245
armas-30-_jpg.rf.59f17a0c15a0e98acabf77885b8c8007.jpg
Time Segment Image : 0.737677
mgd_varying_frame68_jpg.rf.ef2ad947147d790a5c041004724d1067.jpg
Time Segment Image : 0.846110
454744887-1615223010_jpeg_jpg.rf.4c43ec9299cd0504550e013f28a18b96.jpg
Time Segment Image : 0.965516
419159897-1606023748_jpeg_jpg.rf.f1cb74fb2d8d72fc9d9669f8fd602ea3.jpg
Time Segment Image : 1.088058
423637303-1607076161_jpeg_jpg.rf.df58c0cec8182cbd

# Segemtancion partes del cuerpo

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [ ]:
!pip uninstall -y detectron2 
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow_decision_forests
!rm -rf build/ **/*.so

Found existing installation: detectron2 0.6
Uninstalling detectron2-0.6:
  Successfully uninstalled detectron2-0.6
Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Successfully uninstalled tensorflow-2.9.2


In [ ]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 tf_bodypix opencv-python tfjs_graph_converter matplotlib

In [ ]:
!python -m pip install --upgrade imutils

## Clases

In [ ]:
class BodyPix:
  def __init__(self,sourcePath,outputPath) -> None:
    #self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16))
    self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_RESNET50_FLOAT_STRIDE_16))
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def maskImage(self,imagePath):
    imageName = os.path.basename(imagePath)
    
    # load model (once)

    image = tf.keras.utils.load_img(imagePath)
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    # get prediction result
    #image = tf.keras.preprocessing.image.load_img(fullPath)    
    result = self.bodypix_model.predict_single(image_array)

    # simple mask
    mask = result.get_mask(threshold=0.75)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-mask",imageName),
        mask)

    # colored mask (separate colour for each body part)
    colored_mask = result.get_colored_part_mask(mask)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-colored-mask",imageName),
        colored_mask)



    poses = result.get_poses()
    image_with_poses = draw_poses(
        image_array.copy(),  # create a copy to ensure we are not modifing the source image
        poses,
        keypoints_color=(255, 100, 100),
        skeleton_color=(100, 100, 255)
    )
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-poses",imageName),
        image_with_poses
    )
    

  def processImages(self):       
    start = time.time() 
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1
      imageName = os.path.basename(imagePath)
      print(imageName)      
      self.maskImage(imagePath) 
      print("Time Taken Body Pix : %f" % (time.time() - start))

## Liberias BodyPix

In [ ]:
import tensorflow as tf
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths
from tf_bodypix.draw import draw_poses  # utility function using OpenCV

## Creacion proyecto body pix RDN

In [ ]:
!rm -rf body-pix-rdn
!mkdir body-pix-rdn
!mkdir "body-pix-rdn/person"
!mkdir "body-pix-rdn/person/output-mask"
!mkdir "body-pix-rdn/person/output-colored-mask"
!mkdir "body-pix-rdn/person/output-poses"

## Creacion Pipeline red detección partes cuerpo RDN

In [ ]:
sourcePathMask = os.path.join(mainPath,"super-resolution-rdn/person")
outputPathBody = os.path.join(mainPath,"body-pix-rdn/person")
bodyPix = BodyPix(sourcePathMask,outputPathBody)
bodyPix.processImages()

./super-resolution-rdn/person/*.jpg
armas-299-_jpg.rf.7b79e627eca3de74fa325558ecb7c0a1.jpg
Time Taken Body Pix : 4.459844
mgd_custom_frame197_jpg.rf.6b2101e506e1477d6e07509a4d6f51c7.jpg
Time Taken Body Pix : 4.965433
gmd_3_frame84_jpg.rf.a07c5f0c12debf8c9ca921839613e496.jpg
Time Taken Body Pix : 5.735367
armas-263-_jpg.rf.69d306c56b8ce31ae3f7c134245a688f.jpg
Time Taken Body Pix : 6.590691
gmd_1_frame194_jpg.rf.502932c0d41979ec30fdc4455a231b6a.jpg
Time Taken Body Pix : 7.353193
mgd_varying_frame147_jpg.rf.739f0e78220469228c7e068326f3dbd7.jpg
Time Taken Body Pix : 8.732010
armas-30-_jpg.rf.59f17a0c15a0e98acabf77885b8c8007.jpg
Time Taken Body Pix : 9.618219
mgd_varying_frame68_jpg.rf.ef2ad947147d790a5c041004724d1067.jpg
Time Taken Body Pix : 11.076553
454744887-1615223010_jpeg_jpg.rf.4c43ec9299cd0504550e013f28a18b96.jpg
Time Taken Body Pix : 11.923055
419159897-1606023748_jpeg_jpg.rf.f1cb74fb2d8d72fc9d9669f8fd602ea3.jpg
Time Taken Body Pix : 13.100289
423637303-1607076161_jpeg_jpg.rf.df58

# Reemplazo mascara 

## Librerias

In [ ]:
import shutil

## Clases

In [ ]:
class BitMaskGenerator:
  def __init__(self,sourcePath,outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def createBitMask(self,imagePath):
    img = Image.open(imagePath)
    img = img.convert("RGBA")
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255,0,0, 0))
        else:
            newData.append(item)

    img.putdata(newData)
    return img.convert('RGB')
    

    return False,None
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.png"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1                      
      rgb_im = self.createBitMask(imagePath)    
      imageName = os.path.basename(imagePath)      
      print(imageName)
      imageNamePng = Path(imageName).stem+'.png'      
      imagePngPath = os.path.join(self.outputPath,imageNamePng) 
      rgb_im.save(imagePngPath)     
      print("Time BitMaskRed Image : %f" % (time.time() - start))

In [ ]:
class ImageBitMaskUtil:
  def __init__(self,sourcePath,pathBitMask, pathBitRedMask, pathBox,outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    self.pathBitMask = pathBitMask
    self.pathBox = pathBox
    self.pathBitRedMask = pathBitRedMask
    
  def readBox(self,boxPath):
    values = []
    with open(boxPath, "r") as f:
      for line in f:
        values.append(float(line.strip()))
    return values
  def pasteBitMask(self,pathImage):
    imageName = os.path.basename(pathImage)
    name  = Path(imageName).stem
    pathImageMask = os.path.join(self.pathBitMask,name+".png")
    pathBitRedMask = os.path.join(self.pathBitRedMask,name+".png")
    if os.path.exists(pathImageMask):
      print(imageName)      
      boxImagePath  = os.path.join(self.pathBox,name+'.txt')
      box = self.readBox(boxImagePath)      
      print("box",box)
      with Image.open(pathImage) as imageBody:
          imageBody.load()
      with Image.open(pathBitRedMask) as imgGunRedMask:
          imgGunRedMask.load()
      with Image.open(pathImageMask) as imgGunMask:
          imgGunMask.load()
    
      imageBody.paste(
          imgGunRedMask.resize((imgGunRedMask.width, imgGunRedMask.height)),
          (int(box[0]) ,int(box[1])),
          imgGunMask.resize((imgGunMask.width, imgGunMask.height)),
      )
      pathImageResult = os.path.join(self.outputPath,imageName)
      imageBody.save(pathImageResult)
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1               
      imageName = os.path.basename(imagePath)      
      self.pasteBitMask(imagePath)             
      print("Time PasteBitMaskRed into Image : %f" % (time.time() - start))

In [ ]:
class ImageSegmentedUtil:
  def __init__(self,sourcePathSegmented,sourcePath, outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    self.sourcePathSegmented = sourcePathSegmented    
    


  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePathSegmented+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1        
      imageName = os.path.basename(imagePath)
      print(imageName)
      originalImagePath = os.path.join(self.sourcePath,imageName) 
      outputImagePath  = os.path.join(self.outputPath,imageName) 
      shutil.copyfile(originalImagePath, outputImagePath)           
      print("Time PasteBitMaskRed into Image : %f" % (time.time() - start))

## Creacion proyecto merge armas segmentadas y personas

In [ ]:
!rm -rf segmented-body-parts-gun-rdn
!mkdir segmented-body-parts-gun-rdn
!mkdir "segmented-body-parts-gun-rdn/original"
!mkdir "segmented-body-parts-gun-rdn/segmentado"

## Creacion Pipeline procesamiento merge partes cuerpo segmentadas y armas

In [ ]:
sourcePatBinMask = os.path.join(mainPath,"guns-detection-rdn/binmask")
outputPathBinRedMask = os.path.join(mainPath,"guns-detection-rdn/binredmask")
bitMaskGenerator = BitMaskGenerator(sourcePatBinMask,outputPathBinRedMask)
bitMaskGenerator.processImages()

mgd_varying_frame68_jpg.rf.ef2ad947147d790a5c041004724d1067.png
Time BitMaskRed Image : 0.028005
gmd_5_frame62_jpg.rf.32d797d04082909cc03e5ccea71ffc53.png
Time BitMaskRed Image : 0.030861
423637303-1607076161_jpeg_jpg.rf.df58c0cec8182cbd3ee007e32019b2fc.png
Time BitMaskRed Image : 0.124684
Untitled-1-33_jpg.rf.0b36c9b2421fa0c6c3cae40a8b0c69ae.png
Time BitMaskRed Image : 0.134979
gmd_2_frame37_jpg.rf.5baed1bcc40f6e52f503769bef85bde5.png
Time BitMaskRed Image : 0.137139
gmd_1_frame194_jpg.rf.502932c0d41979ec30fdc4455a231b6a.png
Time BitMaskRed Image : 0.139379
armas-263-_jpg.rf.69d306c56b8ce31ae3f7c134245a688f.png
Time BitMaskRed Image : 0.154819
gmd_3_frame84_jpg.rf.a07c5f0c12debf8c9ca921839613e496.png
Time BitMaskRed Image : 0.158298
gmd_2_frame11_jpg.rf.a08c02eba5aba8a2b34e2f20f02746cc.png
Time BitMaskRed Image : 0.160682
419159897-1606023748_jpeg_jpg.rf.f1cb74fb2d8d72fc9d9669f8fd602ea3.png
Time BitMaskRed Image : 0.211826
gmd_2_frame38_jpg.rf.8d3dba4f89d4214bd63a95a0f01787ac.png
Time

In [ ]:
sourcePathBody = "body-pix-rdn/person/output-colored-mask"
pathBitMask = "guns-detection-rdn/binmask"
pathBitRedMask = "guns-detection-rdn/binredmask"
boxPath = "guns-detection-rdn/boxes"
outputPathSegmented = "segmented-body-parts-gun-rdn/segmentado"
imageBitMaskUtil = ImageBitMaskUtil(sourcePathBody,pathBitMask, pathBitRedMask, boxPath,outputPathSegmented)
imageBitMaskUtil.processImages()

armas-299-_jpg.rf.7b79e627eca3de74fa325558ecb7c0a1.jpg
box [618.56805, 572.8567, 657.919, 685.53125]
Time PasteBitMaskRed into Image : 0.026582
Time PasteBitMaskRed into Image : 0.026743
gmd_3_frame84_jpg.rf.a07c5f0c12debf8c9ca921839613e496.jpg
box [300.46854, 412.29004, 362.0, 501.33936]
Time PasteBitMaskRed into Image : 0.038824
armas-263-_jpg.rf.69d306c56b8ce31ae3f7c134245a688f.jpg
box [82.86046, 127.36709, 241.44652, 364.70782]
Time PasteBitMaskRed into Image : 0.058661
gmd_1_frame194_jpg.rf.502932c0d41979ec30fdc4455a231b6a.jpg
box [0.0, 271.2685, 43.450256, 327.71115]
Time PasteBitMaskRed into Image : 0.069099
mgd_varying_frame147_jpg.rf.739f0e78220469228c7e068326f3dbd7.jpg
box [109.62228, 647.5389, 314.2229, 870.8927]
Time PasteBitMaskRed into Image : 0.104504
armas-30-_jpg.rf.59f17a0c15a0e98acabf77885b8c8007.jpg
box [54.358513, 170.9082, 199.12994, 291.6239]
Time PasteBitMaskRed into Image : 0.125618
mgd_varying_frame68_jpg.rf.ef2ad947147d790a5c041004724d1067.jpg
box [919.308, 5

In [ ]:
outputOriginalPath = "segmented-body-parts-gun-rdn/original"
imageSegmentedUtil = ImageSegmentedUtil(outputPathSegmented,sourcePathMask, outputOriginalPath)
imageSegmentedUtil.processImages()

armas-299-_jpg.rf.7b79e627eca3de74fa325558ecb7c0a1.jpg
Time PasteBitMaskRed into Image : 0.000676
gmd_3_frame84_jpg.rf.a07c5f0c12debf8c9ca921839613e496.jpg
Time PasteBitMaskRed into Image : 0.000891
armas-263-_jpg.rf.69d306c56b8ce31ae3f7c134245a688f.jpg
Time PasteBitMaskRed into Image : 0.001103
gmd_1_frame194_jpg.rf.502932c0d41979ec30fdc4455a231b6a.jpg
Time PasteBitMaskRed into Image : 0.001264
mgd_varying_frame147_jpg.rf.739f0e78220469228c7e068326f3dbd7.jpg
Time PasteBitMaskRed into Image : 0.001468
armas-30-_jpg.rf.59f17a0c15a0e98acabf77885b8c8007.jpg
Time PasteBitMaskRed into Image : 0.001695
mgd_varying_frame68_jpg.rf.ef2ad947147d790a5c041004724d1067.jpg
Time PasteBitMaskRed into Image : 0.001932
454744887-1615223010_jpeg_jpg.rf.4c43ec9299cd0504550e013f28a18b96.jpg
Time PasteBitMaskRed into Image : 0.002281
419159897-1606023748_jpeg_jpg.rf.f1cb74fb2d8d72fc9d9669f8fd602ea3.jpg
Time PasteBitMaskRed into Image : 0.010631
423637303-1607076161_jpeg_jpg.rf.df58c0cec8182cbd3ee007e32019b2

## Compresion Archivos

In [ ]:
!rm -rf segmented-body-parts-gun-rdn.tar.gz
!tar -czvf segmented-body-parts-gun-rdn.tar.gz "./segmented-body-parts-gun-rdn"

## Super Resolicon RDN

## Creacion proyecto

In [ ]:
!rm -rf body-pix-rrdn
!mkdir body-pix-rrdn
!mkdir "body-pix-rrdn/person"
!mkdir "body-pix-rrdn/person/output-mask"
!mkdir "body-pix-rrdn/person/output-colored-mask"
!mkdir "body-pix-rrdn/person/output-poses"

## Creacion Pipeline Super Resolucion RDN

In [ ]:
sourcePathMask = os.path.join(mainPath,"super-resolution-rdn/person")
outputPathMask = os.path.join(mainPath,"body-pix/person")
bodyPix = BodyPix(sourcePathMask,outputPathMask)
bodyPix.processImages()

In [ ]:
!rm -rf super-resolution-rdn.tar.gz
!tar -czvf super-resolution-rdn.tar.gz "./super-resolution-rdn/person"

## Cargar en drive resultados

In [ ]:
import os
from google.colab import drive
ROOT_FOLDER='/content/'
ROOT_DRIVE_FOLDER=ROOT_FOLDER+'drive/'
drive.mount(ROOT_DRIVE_FOLDER)

In [ ]:
!cp --suffix=_v_$(date +"%y-%m-%d--%T")  ./super-resolution-esrgan-personas-armadas.tar.gz /content/drive/MyDrive/tmf-person-data/data_set_processed/personas-armadas-detectadas

In [ ]:
!cp --suffix=_v_$(date +"%y-%m-%d--%T")  ./super-resolution-rdn.tar.gz /content/drive/MyDrive/tmf-person-data/data_set_processed/personas-armadas-detectadas